In [113]:
!pip install datasets transformers
!pip install evaluate
!pip install rouge_score
!pip install -U bitsandbytes
!pip install faiss-cpu 
!pip install langchain
!pip install langchain_community

In [150]:
from datasets import load_dataset
from transformers import PreTrainedTokenizerBase
from transformers import AutoTokenizer, DPRContextEncoder,DPRContextEncoderTokenizerFast, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer, pipeline
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration, pipeline, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from datasets import DatasetDict
import evaluate
import torch
import faiss
from datasets import Dataset
from langchain.schema    import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings   import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms         import HuggingFacePipeline
from langchain.chains       import RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import math
import re
squad  = evaluate.load("squad")
rouge  = evaluate.load("rouge")
bleu   = evaluate.load("bleu")

## Helper functions

In [116]:
def extract_ans(text: str) -> str:
    """
    Return whatever follows '####' up to the first newline
    or up to non-answer characters like '-' . 
    """
    m = re.search(r"####\s*([^\n\-]+)", text)   # stops before newline or '-'
    if m:
        return m.group(1).strip()
    # fallback: last non-empty line
    for line in reversed(text.strip().splitlines()):
        if line.strip():
            return line.strip()
    return text.strip()

In [117]:
def preprocess(examples, tokenizer: PreTrainedTokenizerBase):
    tokenized = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=256,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized.pop("offset_mapping")
    start_positions = []
    end_positions = []
    for i, offsets in enumerate(offset_mapping):
        sample_idx = sample_mapping[i]
        answers = examples["answers"][sample_idx]
        seq_ids = tokenized.sequence_ids(i)
        cls_index = 0
        start_positions.append(cls_index)
        end_positions.append(cls_index)
        if len(answers["answer_start"]) == 0:
            continue
        start_char = answers["answer_start"][0]
        end_char = start_char + len(answers["text"][0])
        context_tokens = [idx for idx, s_id in enumerate(seq_ids) if s_id == 1]
        if not context_tokens:
            continue
        chunk_start_char = offsets[context_tokens[0]][0]
        chunk_end_char = offsets[context_tokens[-1]][1]
        if not (chunk_start_char <= start_char and end_char <= chunk_end_char):
            continue
        token_start_index = cls_index
        token_end_index = cls_index
        for idx in context_tokens:
            off_start, off_end = offsets[idx]
            if off_start <= start_char < off_end:
                token_start_index = idx
            if off_start < end_char <= off_end:
                token_end_index = idx
        start_positions[-1] = token_start_index
        end_positions[-1] = token_end_index
    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions
    return tokenized


In [118]:
def print_metrics(ds, custom_pipeline):
    #Run the pipeline
    outputs = custom_pipeline(question=ds["question"], context=ds["context"])

    #Prepare for SQuAD EM/F1
    preds = [
        {"id": ex["id"], "prediction_text": out["answer"]}
        for ex, out in zip(ds, outputs)
    ]
    refs = [
        {"id": ex["id"], "answers": ex["answers"]}
        for ex in ds
    ]
    squad_res = squad.compute(predictions=preds, references=refs)

    #Prepare raw text lists for Rouge & BLEU
    pred_texts = [out["answer"] for out in outputs]
    ref_texts  = [ex["answers"]["text"][0] for ex in ds]

    rouge_res = rouge.compute(predictions=pred_texts, references=ref_texts)
    bleu_res  = bleu.compute(
        predictions=pred_texts,
        references=[[r] for r in ref_texts]
    )

    #Print all metrics
    print(
        f"EM: {squad_res['exact_match']:.2f}, "
        f"F1: {squad_res['f1']:.2f}, "
        f"Rouge-L: {rouge_res['rougeL'] * 100:.2f}, "
        f"BLEU: {bleu_res['bleu'] * 100:.2f}"
    )

## Dataset loading

In [119]:
raw = load_dataset("squad")
sub = raw["train"].shuffle(seed=42).select(range(20000))
split = sub.train_test_split(test_size=0.2, seed=42)

test_subset = raw["validation"].shuffle(seed=42).select(range(2_000))
data = DatasetDict({
    "train":      split["train"],
    "validation": split["test"],
    "test":       test_subset
})

## Experiment 1

In [ ]:
model_name = "distilbert-base-uncased-distilled-squad"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model_distilbert  = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
tokenized = data.map(
    lambda ex: preprocess(ex, tokenizer),
    batched=True,
    remove_columns=raw["train"].column_names
)

In [ ]:
args = TrainingArguments(
  output_dir="distilbert-qa",
  per_device_train_batch_size=128,
  per_device_eval_batch_size=128,
  num_train_epochs=10,
  learning_rate=3e-5,
  weight_decay=0.01,
  logging_steps=200,
  log_level="info",
  report_to=["none"],       
  disable_tqdm=False,      
  fp16=True if torch.cuda.is_available() else False,            
)

In [ ]:
trainer_full = Trainer(
  model=model_distilbert,
  args=args,
  train_dataset=tokenized["train"],
  eval_dataset =tokenized["validation"],
  processing_class=tokenizer,
  compute_metrics= print_metrics
)
trainer_full.train()

In [ ]:
qa = pipeline(
    "question-answering",
    model="/kaggle/working/distilbert-qa/checkpoint-1740",
    tokenizer="/kaggle/working/distilbert-qa/checkpoint-1740",
    device=0,
)

ds = data["test"]
print_metrics(ds, qa)

## Experiment 2

In [ ]:
model_partial = AutoModelForQuestionAnswering.from_pretrained(model_name)

#### Partial fine tuning

In [ ]:
for param in model_partial.distilbert.parameters():
    param.requires_grad = False


In [ ]:
trainer_partial = Trainer(
    model=model_partial,
    args=args_partial,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
)
trainer_partial.train()

In [ ]:
qa_partial = pipeline(
    "question-answering",
    model="distilbert-qa-partial/checkpoint-417",  
    tokenizer="distilbert-qa-partial/checkpoint-417",
    device=0 if torch.cuda.is_available() else -1,
)

print_metrics(data["test"], qa_partial)

## Experiment 3

In [131]:
contexts = raw["train"]["context"] + raw["validation"]["context"]
unique   = list(dict.fromkeys(contexts))
docs     = [Document(page_content=c) for c in unique]


#we need smaller pieces to fit the model 
#also it improves semantic search accuracy as smaller chunks allow the faiss index to capture a more specific meaning 
#chunk overlap preserves context
splitter    = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs  = splitter.split_documents(docs)


#Building an embedding for each chunk
embeddings = HuggingFaceEmbeddings()

#Where we store the chunk and the corresponding embedding
#Faiss index contains vectors 
db         = FAISS.from_documents(split_docs, embeddings)
#What will help us retrieve the 4 most relevant using the index inialized in the previous line
retriever  = db.as_retriever(search_kwargs={"k": 4})

/tmp/ipykernel_35/235387335.py:14: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/12e86a3c702fc3c50205a8db88f0ec7c0b6b94a0/config.json
Model config MPNetConfig {
  "architectures": [
    "MPNetForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "mpnet",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "relative_attention_num_buckets": 32,
  "transform

In [132]:
MODEL_ID = "google/flan-t5-large"          
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_ID,
    device_map="auto",           
)

qa_pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    max_new_tokens=64,
)

llm = HuggingFacePipeline(pipeline=qa_pipe)

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--google--flan-t5-large/snapshots/0613663d0d48ea86ba8cb3d7a44f0f65dc596a2a/spiece.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-large/snapshots/0613663d0d48ea86ba8cb3d7a44f0f65dc596a2a/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-large/snapshots/0613663d0d48ea86ba8cb3d7a44f0f65dc596a2a/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-large/snapshots/0613663d0d48ea86ba8cb3d7a44f0f65dc596a2a/tokenizer_config.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-large/snapshots/0613663d0d48ea86ba8cb3d7a44f0f65dc596a2a/config.json
Model config T5Config {
  "

In [ ]:
zero_shot_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "{context}\n\n"
        "Question: {question}\n"
        "Answer:"
    ),
)

zero_shot_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": zero_shot_prompt},
)

In [133]:
cot_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Example:\n"
        "Passages:\n2 + 2 equals four.\n\n"
        "Question: What is 2 + 2?\n"
        "Step-by-step reasoning:\n"
        "1. Two plus two equals four.\n"
        "#### 4\n"
        "----\n\n"
        "Use the following passages to answer the question.\n"
        "Think step-by-step. When you’re done, write\n"
        "#### <answer> on its own line.\n\n"
        "Passages:\n{context}\n\n"
        "Question: {question}\n\n"
        "Step-by-step reasoning:\n1. "
    ),
)

cot_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",          # or "map_reduce" if you prefer
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": cot_prompt},
)

In [ ]:
def print_chain_metrics(chain, data, apply_extract: bool = True):
    preds, refs = [], []
    rouge_hyp, rouge_ref, bleu_refs = [], [], []

    total = len(data["test"])
    for idx, ex in enumerate(data["test"], 1):
        print(f"[{idx}/{total}] Processing QID={ex['id']}…")

        raw = chain({"query": ex["question"]})["result"]
        
        text = extract_ans(raw) if apply_extract else raw.strip()                    

        preds.append({"id": ex["id"], "prediction_text": text})
        refs.append({
            "id": ex["id"],
            "answers": {
                "text": ex["answers"]["text"],
                "answer_start": ex["answers"]["answer_start"]
            }
        })
        rouge_hyp.append(text)
        rouge_ref.append(ex["answers"]["text"][0])
        bleu_refs.append(ex["answers"]["text"])

        print(f"[{idx}/{total}] Done.\n")

    print("All examples processed — now computing metrics…\n")

    s_res = squad.compute(predictions=preds, references=refs)
    r_res = rouge.compute(predictions=rouge_hyp, references=rouge_ref)
    b_res = bleu.compute(predictions=rouge_hyp, references=bleu_refs)

    print(f"EM:           {s_res['exact_match']:.2f}")
    print(f"F1:           {s_res['f1']:.2f}")
    print(f"ROUGE-L F1:   {r_res['rougeL'] * 100:.2f}")
    print(f"BLEU:         {b_res['bleu'] * 100:.2f}")


In [ ]:
print_chain_metrics(cot_chain, data,False)

In [ ]:
print_chain_metrics(zero_shot_chain, data, apply_extract=False)

In [ ]:
# N = 5   # how many examples to inspect

# for i in range(N):
#     ex       = data["test"][i]           # one SQuAD example
#     question = ex["question"]

#     # ❱❱  feed the question as {"query": ...}
#     res = cot_chain({"query": question})   # or {"question": question} if you changed input_key
#     raw = res["result"]                    # chain output (reasoning + answer)
#     final = raw.split("Final Answer:")[-1].strip()

#     print(f"\n=== Example {i+1}/{N} ===")
#     print("QUESTION:", question)
#     print("RAW COT OUTPUT:\n", raw)
#     print("EXTRACTED ANSWER:", final)
#     print("GOLD ANSWERS:", ex["answers"]["text"])

### Chat Bot with memory

In [182]:
memory = ConversationBufferWindowMemory(
    k=6,           
    ai_prefix="Assistant",
    human_prefix="User",
    return_messages=True,
)
chatbot = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False,   
)
messages = [
    "Hi there!",
    "I’m Ahmed.",
    "What’s my name?",
    "Who am I chatting with?"
]

for msg in messages:
    reply = chatbot.predict(input=msg)
    print(f"> You: {msg}")
    print(f"> Bot: {reply}\n")
print("=== Memory Buffer ===")
print(chatbot.memory.buffer)

Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


> You: Hi there!
> Bot: Hello!



Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


> You: I’m Ahmed.
> Bot: Ahmed is a human.



Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


> You: What’s my name?
> Bot: Ahmed is my name.

> You: Who am I chatting with?
> Bot: Ahmed

=== Memory Buffer ===
[HumanMessage(content='Hi there!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}), HumanMessage(content='I’m Ahmed.', additional_kwargs={}, response_metadata={}), AIMessage(content='Ahmed is a human.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What’s my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Ahmed is my name.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Who am I chatting with?', additional_kwargs={}, response_metadata={}), AIMessage(content='Ahmed', additional_kwargs={}, response_metadata={})]


## Adding retrieval to conversational chain

In [185]:
memory = ConversationBufferMemory(
  memory_key="chat_history",
  return_messages=True
)
conv_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    # return_source_documents=True,   
)

In [186]:
print(conv_chain( {"question" : "where is paris?" } )["answer"])
print(conv_chain( {"question" : "what did I just ask?" } )["answer"])

Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


northern central France


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


b).


## Adding prompting to conversational retrieval chain

In [187]:
CONDENSE_PROMPT = PromptTemplate(
    input_variables=["chat_history","question"],
    template=(
        "The following is a conversation between User and Assistant:\n"
        "{chat_history}\n\n"
        "Given that, rewrite the user's last question\n"
        "so that it can be understood without context.\n\n"
        "Follow-up question: {question}\n"
        "Standalone question:"
    ),
)
QA_PROMPT = PromptTemplate(
    input_variables=["context","question"],
    template=(
        "You are an expert assistant. Use the following passages:\n\n"
        "{context}\n\n"
        "to answer the question as concisely as possible.\n\n"
        "Question: {question}\n"
        "Answer:"
    ),
)
conv_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    condense_question_prompt=CONDENSE_PROMPT,
    combine_docs_chain_kwargs={"prompt": QA_PROMPT},
)
print(conv_chain( {"question" : "where is paris?" } )["answer"])
print(conv_chain( {"question" : "tell me about my last question" } )["answer"])

Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


northern central France


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


northern central France
